In [50]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from transformers import *

In [51]:
# the data here needs to be named properly (in english)
# converted to proper dtypes, imputed, encoded and then merged
raw_houses = pd.read_csv("data/raw_houses.csv")
raw_sells = pd.read_csv("data/raw_sells.csv")

In [52]:
# parse inconsistent floats, booleans, convert to proper dtypes
def apply_dtype_trfmer(
  X: pd.DataFrame, features: pd.DataFrame
) -> pd.DataFrame:
  int_ft = features.index[features["Dtype"] == "int"]
  float_ft = features.index[features["Dtype"] == "float"]
  datetime_ft = features.index[features["Dtype"] == "datetime"]
  boolean_ft = features.index[features["Dtype"] == "boolean"]

  dtype_trfmer = ColumnTransformer(
    [
      ("int_trfm", IntTransformer(), int_ft),
      ("float_trfm", FloatTransformer(), float_ft),
      ("datetime_trfm", DatetimeTransformer(), datetime_ft),
      ("boolean_trfm", BooleanTransformer(), boolean_ft),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
  )
  dtype_trfmer.set_output(transform="pandas")
  return dtype_trfmer.fit_transform(X)

In [53]:
# initial df was transposes and had houses as columns
houses = raw_houses.transpose()
houses.columns = houses.iloc[0]
houses.index = pd.RangeIndex(0, len(houses.index))

# rename columns, convert to proper dtypes
ft_houses = pd.read_csv("data/features_houses.csv", index_col="Old")
houses = houses[1:].rename(columns=ft_houses["New"]).reset_index(drop=True)
ft_houses = ft_houses.reset_index().set_index("New")
houses = apply_dtype_trfmer(houses, ft_houses)

In [54]:
# rename columns, convert to proper dtypes
ft_sells = pd.read_csv("data/features_sells.csv", index_col="Old")
sells = raw_sells.rename(columns=ft_sells["New"])
ft_sells = ft_sells.reset_index().set_index("New")
sells = apply_dtype_trfmer(sells, ft_sells)

In [55]:
# need to merge two tables using sells["HouseId", "HouseName"] and houses[["Name"]]
# some houses["Name"] are in form of "{Name or Address} {HouseId}"
# but a few of them doesn't have {HouseId}
# HouseId >= 3062 if exists so we can clearly separate it from the first part
def pop_project_id(name: str) -> tuple[str, int]:
  sep = name.rfind(" ")
  if sep == -1:
    return name, pd.NA

  try:
    project_id = int(name[sep + 1 :])
    if project_id < 3062:
      raise Exception
  except:
    return name, pd.NA

  return name[:sep], project_id


# https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
houses["HouseName"], houses["HouseId"] = zip(
  *houses["Name"].map(pop_project_id)
)
houses["HouseId"] = houses["HouseId"].astype("Int32")

In [56]:
# drop uninstresting data
houses_drops = (
  ["Name", "SeaView", "InfoDate", "SoldFlatsRubl", "NSoldFlats"]
  + ["NSoldParkSlots", "SoldFlatsArea", "SoldPercent"]
  + ["MeanSqMeterCost", "NSoldNonresid"]
)
houses.drop(columns=houses_drops, inplace=True, errors="ignore")

# predict only Vladivostok real estate prices
if "Settlement" in sells.columns:
  sells = sells[sells["Settlement"] == "Владивосток"].reset_index(drop=True)
# sum up sells for each house over months
sells_groupby = [
  # "ProjectId",  # doesn't appear in houses.Name
  "HouseId",
  "CompanyName",
  "HouseName",
  "HouseCatg",
  "HouseStatus",
]
# perhaps columns like SoldNonresid, SoldParkSlots cannot be predicting features
sells_cols = sells_groupby + ["SoldFlatsArea", "SoldFlatsRubl"]
# sum over aug 2021-aug 2022
sells = sells[sells_cols].groupby(by=sells_groupby, as_index=False).sum()

# target
# avoid division by zero: pd.NA propagates
sells["SoldFlatsArea"] = (
  sells["SoldFlatsArea"]
  .map(lambda x: np.nan if x == 0.0 else x)
  .astype("Float32")
)
sells["SqMeterCost"] = sells["SoldFlatsRubl"] / sells["SoldFlatsArea"]

In [57]:
# cant believe in SqMeterCost == 32148
def drop_outliers(sells: pd.DataFrame, tail: float) -> pd.DataFrame:
  lower = sells["SqMeterCost"].quantile(tail, interpolation="lower")
  higher = sells["SqMeterCost"].quantile(1 - tail, interpolation="higher")
  return sells[
    (sells["SqMeterCost"] >= lower) & (sells["SqMeterCost"] <= higher)
  ]


# unreliable data
if sells["SqMeterCost"].min() < 50000:
  clean_sells = drop_outliers(sells, 0.1)

In [ ]:
# map NA houses.HouseId to valid sells.HouseId
na_mapping = {
  "Садгород-357": 44295,
  "Садгород-295": 44295,
  # 'Времена года': -1,     # wtf
  "Восточный ЛУЧ-5": [
    37381,
    37701,
    37703,
    37704,
    37705,
    34275,
    37333,
    36352,
  ],
  "Новые горизонты": [40959, 42989],
  "Басаргина, д. 2": 41333,
  "Басаргина, д. 2, б/с 2 10 эт": 41422,
  "Басаргина, д. 2, б/с 2 18 эт": 41487,
  "Борисенко, д. 100, лит. Е": [38128, 38129],
  "Изумрудный, 1оч": [13283, 13284, 13285, 37526, 37527],
}

In [60]:
houses[houses["HouseId"].isna()]["HouseName"].to_list()

['Садгород-357',
 'Садгород-295',
 'Времена года',
 'Восточный ЛУЧ-5',
 'Восточный ЛУЧ-6',
 'Восточный ЛУЧ-7',
 'Восточный ЛУЧ-8',
 'Восточный ЛУЧ-9',
 'Восточный ЛУЧ-10',
 'Новые горизонты',
 'Басаргина, д. 2',
 'Басаргина, д. 2, б/с 2 10 эт',
 'Басаргина, д. 2, б/с 2 18 эт',
 'Борисенко, д. 100, лит. Е',
 'Изумрудный, 1оч']